In [1]:

import re
import os
import sys
from glob import iglob
import json

#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"
import random
import pandas as pd

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)



In [140]:
klikk_inp = '../klikkek/data/results/cl_final_egt200_egt07.txt'
mn_fn_db_file = '../data/mini_corpus/mn_fn_db.tsv'
valid_output_file = '../klikkek/data/results/symdiffs/cl_final_egt200_egt07_1_1.txt'
valid_output_file2 = '../klikkek/data/results/symdiffs/cl_final_egt200_egt07_1_1_fullos.txt'

excel_output_file = '../klikkek/data/results/symdiffs/cl_final_egt200_egt07_cliques.xlsx'

cooccur_threshold = 1
noun_freq_threshold = 1

In [141]:
mn_fn_db = pd.read_csv(mn_fn_db_file, sep='\t')
mn_fn_db_filt = mn_fn_db[mn_fn_db['count'] >= cooccur_threshold]

mn_fn_db_filt.head()

,mn,fn,count
0,%-nyi,mennyiség,1
1,%-nyi,részvény,1
2,%-nyi,kedvezmény,2
3,%-os,KDNP,1
4,%-os,adókulcs,1


In [142]:
def klikk_processor(klikk_inp):
    data = {}
    with open(klikk_inp) as inp:
        data = json.load(inp)
    return data

def get_klikk_data_table(klikk_inp_file):
    
    klikkdata = klikk_processor(klikk_inp_file)
    new_klikk_data_ls = []
    klikk_id = 0 # Melyik mn_névről van szó
    klikk_group_id = 0 #Melyik mn, melyik klikkjéről van szó
    for k, v in klikkdata.items():
        #k, v
        for i, klikk_items in enumerate(v):
            #i, klikk_items
            for klikk_elem in klikk_items:
                new_record = [k, klikk_id, klikk_elem, klikk_group_id, klikk_inp]
                new_klikk_data_ls.append(new_record)
            klikk_group_id+=1
        klikk_id+=1
    klikk_data_table = pd.DataFrame(new_klikk_data_ls)
    klikk_data_table.columns = ['klikk_name', 'klikk_id', 'klikk_elem','klikk_group_id', 'source_file']
    return klikk_data_table
    

def get_klikk_group_size(klikk_data_table):
    
    klikk_group_size = klikk_data_table[['klikk_id', 'klikk_group_id', 
                  'klikk_elem']].groupby(by=['klikk_id', 'klikk_group_id']).count().reset_index()[['klikk_group_id', 'klikk_elem']]
    klikk_group_size.rename({'klikk_elem': 'klikk_group_size'}, axis=1, inplace=True)
    return klikk_group_size

def get_klikk_group_representative_fns(raw_sets, klikk_group_size):
    """
    Legyártjuk az olyan főnevek listáját, amelyek mindegyik, klikk-csoportot alkotó mn-vekkel együtt szerepelnek
    """
    klikk_group_fn_freq = raw_sets[['klikk_name', 'klikk_id', 'klikk_group_id', 'fn', 'mn']].groupby(by=['klikk_name', 'klikk_id', 'klikk_group_id', 'fn']).count().reset_index()
    klikk_group_fn_freq.rename({'mn':'fn_count'}, inplace=True, axis=1)

    # 
    klikk_group_fn_freq_klikk_groups_size = klikk_group_fn_freq[['klikk_group_id','fn', 'fn_count']].merge(klikk_group_size, how = 'left', left_on = 'klikk_group_id', right_on= 'klikk_group_id')
    klikk_group_fn_freq_klikk_groups_size_filt = klikk_group_fn_freq_klikk_groups_size[klikk_group_fn_freq_klikk_groups_size.fn_count == klikk_group_fn_freq_klikk_groups_size.klikk_group_size]

    return klikk_group_fn_freq_klikk_groups_size_filt

def get_klikk_descriptors(raw_sets, klikk_group_fn_freq_klikk_groups_size_filt):

    raw_sets_filt = raw_sets.merge(klikk_group_fn_freq_klikk_groups_size_filt, how = 'inner', left_on = ['klikk_group_id', 'fn'], right_on = ['klikk_group_id', 'fn'])
    raw_sets_filt_klikk_group_descr = raw_sets_filt[['klikk_name', 'klikk_id', 'klikk_group_id', 'fn']].drop_duplicates()
    raw_sets_aggs = raw_sets_filt_klikk_group_descr.groupby(by=['klikk_name', 'klikk_id', 'fn']).count().reset_index()
    klikk_leiro_fn = raw_sets_aggs[raw_sets_aggs.klikk_group_id == 1].copy()
#    klikk_leirok = raw_sets.merge(klikk_leiro_fn[['klikk_name', 'klikk_id', 'fn']], how = 'inner', 
#                   left_on = ['klikk_name', 'klikk_id', 'fn'], right_on = ['klikk_name', 'klikk_id', 'fn'])
    klikk_leiro_fn.rename( {'klikk_group_id' : 'count_klikk_groups'}, axis = 1, inplace=True)
    return klikk_leiro_fn, raw_sets_filt

In [143]:
klikk_data_table = get_klikk_data_table(klikk_inp) # beolvassuk az adatokat
klikk_group_size = get_klikk_group_size(klikk_data_table) # melyik klikk mekkora

raw_sets = klikk_data_table.merge(mn_fn_db_filt, how='left', left_on = 'klikk_elem', right_on = 'mn')

#raw_sets.to_excel('../results/raw_sets.xlsx', index=False)
klikk_group_fn_freq_klikk_groups_size_filt = get_klikk_group_representative_fns(raw_sets, klikk_group_size)

klikk_leiro_fn, raw_sets_filt = get_klikk_descriptors(raw_sets, klikk_group_fn_freq_klikk_groups_size_filt)


klikk_leiro_fn.sort_values(by = 'klikk_id')



,klikk_name,klikk_id,fn,count_klikk_groups
20747,jó,0,osztályzat,1
20495,jó,0,kifogás,1
20494,jó,0,kifejezés,1
20493,jó,0,kiegészítő,1
20492,jó,0,kiegészítés,1
...,...,...,...,...
22968,kilónyi,814,smink,1
22969,kilónyi,814,szemét,1
22970,kilónyi,814,súly,1
22962,kilónyi,814,arany,1


In [144]:
klikk_data_table
klikk2klikkgroup = klikk_data_table[['klikk_group_id', 'klikk_id', 'klikk_name']].drop_duplicates()

klikk2klikkgroup

,klikk_group_id,klikk_id,klikk_name
0,0,0,jó
2,1,0,jó
4,2,1,politikai
6,3,1,politikai
9,4,1,politikai
...,...,...,...
7670,2478,812,kazincbarcikai
7672,2479,813,szürreális
7675,2480,813,szürreális
7679,2481,814,kilónyi


In [145]:
# Melyik klikkben, milyen főnevek a uniq-ok. 
#klikk_leiro_fn
len(klikk_group_fn_freq_klikk_groups_size_filt)

output = klikk_group_fn_freq_klikk_groups_size_filt.merge(klikk2klikkgroup, how = 'inner', left_on = 'klikk_group_id', right_on = 'klikk_group_id').\
merge(klikk_leiro_fn[['klikk_id' ,'fn']], left_on = ['klikk_id', 'fn'], right_on = ['klikk_id', 'fn']).sort_values(by=['klikk_name'])


#output.to_excel(excel_output_file, index=False)

#klikk2klikkgroup.merge(klikk_leiro_fn, how='inner', 
#                       left_on = 'klikk_id', 
#                       right_on = 'klikk_id')


# Klikkek kiiratása

In [146]:

klikk_data_table.head()
klikk.head()
klikk_group.head()

len(klikk_ids)
len(klikk_data_table.klikk_id.unique())

815

In [147]:
# Ebben van, hogy melyik klikknek mi a neve
klikk = klikk2klikkgroup[['klikk_id', 'klikk_name']].drop_duplicates()
# Ebben van, ogy melyik klikkalcsoportnak mi a neve
klikk_group = klikk_data_table[['klikk_group_id', 'klikk_elem']].drop_duplicates()

klikk_ids = list(output['klikk_id'].unique())
klikk_group_id = list(output['klikk_group_id'].unique())
len(klikk_ids)

with open(valid_output_file, 'w') as fout:
    #act_klikk_id = 243
    
    for act_klikk_id in klikk_ids:
        klikk_name = str(klikk[klikk.klikk_id == act_klikk_id].klikk_name.values[0])
        klikk_group_ids = list(output[output.klikk_id == act_klikk_id]['klikk_group_id'].unique())
        for act_klikk_group_id in klikk_group_ids:
            klikk_group_descr = list(klikk_group[klikk_group.klikk_group_id == act_klikk_group_id]['klikk_elem'])
            klikk_group_fns = list(output[output.klikk_group_id == act_klikk_group_id]['fn'])
            fout.write('{0} : {1}'.format(klikk_name, ', '.join(klikk_group_descr)) + '\n')
            fout.write( ', '.join(klikk_group_fns) + '\n')
        fout.write('\n')


## Mindent kiírunk, amit csak lehet


In [148]:
# Ebben van, hogy melyik klikknek mi a neve
klikk = klikk2klikkgroup[['klikk_id', 'klikk_name']].drop_duplicates()
# Ebben van, ogy melyik klikkalcsoportnak mi a neve
klikk_group = klikk_data_table[['klikk_group_id', 'klikk_elem']].drop_duplicates()

klikk_ids = list(klikk_data_table['klikk_id'].unique())
klikk_group_id = list(klikk_data_table['klikk_group_id'].unique())


with open(valid_output_file2, 'w') as fout:
    #act_klikk_id = 243
    
    for act_klikk_id in klikk_ids:
        klikk_name = str(klikk[klikk.klikk_id == act_klikk_id].klikk_name.values[0])
        klikk_group_ids = list(klikk_data_table[klikk_data_table.klikk_id == act_klikk_id]['klikk_group_id'].unique())
        
        #klikk_group_ids = list(output[output.klikk_id == act_klikk_id]['klikk_group_id'].unique())
        for act_klikk_group_id in klikk_group_ids:
            klikk_group_descr = list(klikk_group[klikk_group.klikk_group_id == act_klikk_group_id]['klikk_elem'])
            klikk_group_fns = list(output[output.klikk_group_id == act_klikk_group_id]['fn'])
            fout.write('{0} : {1}'.format(klikk_name, ', '.join(klikk_group_descr)) + '\n')
            fout.write( ', '.join(klikk_group_fns) + '\n')
            fout.write('\n')
        fout.write('____________________________\n\n\n')

In [149]:
raw_sets


,klikk_name,klikk_id,klikk_elem,klikk_group_id,source_file,mn,fn,count
0,jó,0,jó,0,../klikkek/data/results/cl_final_egt200_egt07.txt,jó,2-szer,1.0
1,jó,0,jó,0,../klikkek/data/results/cl_final_egt200_egt07.txt,jó,4-es,1.0
2,jó,0,jó,0,../klikkek/data/results/cl_final_egt200_egt07.txt,jó,99%,1.0
3,jó,0,jó,0,../klikkek/data/results/cl_final_egt200_egt07.txt,jó,AS|As,1.0
4,jó,0,jó,0,../klikkek/data/results/cl_final_egt200_egt07.txt,jó,AZ,1.0
...,...,...,...,...,...,...,...,...
2321445,kilónyi,814,kilós,2482,../klikkek/data/results/cl_final_egt200_egt07.txt,kilós,tömörgumi,15.0
2321446,kilónyi,814,kilós,2482,../klikkek/data/results/cl_final_egt200_egt07.txt,kilós,csuka,18.0
2321447,kilónyi,814,kilós,2482,../klikkek/data/results/cl_final_egt200_egt07.txt,kilós,súly,21.0
2321448,kilónyi,814,kilós,2482,../klikkek/data/results/cl_final_egt200_egt07.txt,kilós,platformcipő,30.0


In [151]:
klikk_group_fn_freq_klikk_groups_size_filt

raw_sets_filt = raw_sets.merge(klikk_group_fn_freq_klikk_groups_size_filt,
                               how = 'inner', left_on = ['klikk_group_id', 'fn'], right_on = ['klikk_group_id', 'fn'])


len(raw_sets)
len(raw_sets_filt)

#raw_sets_filt.head()
raw_sets_filt[raw_sets_filt.fn == 'Magyarország']

raw_sets_filt_g = raw_sets_filt[['klikk_id', 'klikk_group_id', 'fn']].drop_duplicates()

raw_sets_filt_g_counts = raw_sets_filt_g.groupby(by = ['klikk_id', 'fn']).count().reset_index()
raw_sets_filt_g_counts[raw_sets_filt_g_counts.klikk_group_id == 1]
raw_sets_filt_g[raw_sets_filt_g.fn == 'Magyarország']


,klikk_id,klikk_group_id,fn
1084,0,0,Magyarország
11960,4,17,Magyarország
35704,11,48,Magyarország
36676,12,52,Magyarország
38401,13,58,Magyarország
47126,20,80,Magyarország
47272,21,82,Magyarország
48062,22,84,Magyarország
56167,26,96,Magyarország
56275,26,97,Magyarország


In [ ]:
# ezek a főnevek azt tudják, hogy minden klik-ben csak 1x fordulnak elő
klikk_csoport_fns = raw_sets[['klikk_id', 'klikk_group_id', 'fn']].drop_duplicates()

klikk_csoport_fns[klikk_csoport_fns.fn.isin(klikk_leiro_fn['fn'])].groupby(by=['klikk_id', 'fn']).count()



: 

In [154]:
klikk_data_table = get_klikk_data_table(klikk_inp)

# Kiegészítjük a csoport mérettel, hogy tudjuk, melyik klikk-csoportban mennyi főnevet kell várni.
klikk_group_size = klikk_data_table[['klikk_id', 'klikk_group_id', 
                  'klikk_elem']].groupby(by=['klikk_id', 'klikk_group_id']).count().reset_index()[['klikk_group_id', 'klikk_elem']]
klikk_group_size.rename({'klikk_elem': 'klikk_group_size'}, axis=1, inplace=True)
#klikk_data_table = klikk_data_table.merge(klikk_group_size, how='left', left_on = 'klikk_group_id', right_on = 'klikk_group_id')
#klikk_data_table



In [155]:
# Berakjuk a főneveket
raw_sets = klikk_data_table.merge(mn_fn_db_filt, how='left', left_on = 'klikk_elem', right_on = 'mn')
#raw_sets.head()



In [156]:
# megadja nekünk, hogy melyik főnév, melyik csoportban hányszor szerepel
klikk_group_fn_freq = raw_sets[['klikk_name', 'klikk_id', 'klikk_group_id', 'fn', 'mn']].groupby(by=['klikk_name', 'klikk_id', 'klikk_group_id', 'fn']).count().reset_index()
klikk_group_fn_freq.rename({'mn':'fn_count'}, inplace=True, axis=1)

# 
klikk_group_fn_freq_klikk_groups_size = klikk_group_fn_freq[['klikk_group_id','fn', 'fn_count']].merge(klikk_group_size, how = 'left', left_on = 'klikk_group_id', right_on= 'klikk_group_id')
klikk_group_fn_freq_klikk_groups_size_filt = klikk_group_fn_freq_klikk_groups_size[klikk_group_fn_freq_klikk_groups_size.fn_count == klikk_group_fn_freq_klikk_groups_size.klikk_group_size]




In [157]:
# Most kiválasztjuk azokat főneveket és csoportokat, amelyek megfelelnek a fenti kritériumnak

raw_sets_filt = raw_sets.merge(klikk_group_fn_freq_klikk_groups_size_filt, how = 'inner', left_on = ['klikk_group_id', 'fn'], 
                               right_on = ['klikk_group_id', 'fn'])

raw_sets_filt_klikk_group_descr = raw_sets_filt[['klikk_name', 'klikk_id', 'klikk_group_id', 'fn']].drop_duplicates()
raw_sets_aggs = raw_sets_filt_klikk_group_descr.groupby(by=['klikk_name', 'klikk_id', 'fn']).count().reset_index()
klikk_leiro_fn = raw_sets_aggs[raw_sets_aggs.klikk_group_id == 1]

#klikk_leirok = raw_sets.merge(klikk_leiro_fn[['klikk_name', 'klikk_id', 'fn']], how = 'inner', 
#               left_on = ['klikk_name', 'klikk_id', 'fn'], right_on = ['klikk_name', 'klikk_id', 'fn'])

#klikk_leirok
#klikk_leirok.to_excel('../results/klikk_fn_egyszer.xlsx', index=False)


,klikk_name,klikk_id,fn,klikk_group_id
0,abszurd,243,humor,1
1,abszurd,243,mód,1
4,akut,509,veseelégtelenség,1
5,albán,78,csapat,1
6,alelnöki,364,poszt,1
...,...,...,...,...
6474,ötéves,234,gyerek,1
6475,ötéves,234,kisfiú,1
6478,üzbég,596,határ,1
6479,ősi,102,erő,1


In [158]:

raw_sets_aggs = raw_sets_filt[['klikk_name', 'klikk_id', 'fn', 'mn']].drop_duplicates().groupby(by=['klikk_name', 'klikk_id', 'fn']).count().reset_index()
klikk_leiro_fn = raw_sets_aggs[raw_sets_aggs.mn == 1]

klikk_leirok = raw_sets.merge(klikk_leiro_fn[['klikk_name', 'klikk_id', 'fn']], how = 'inner', 
               left_on = ['klikk_name', 'klikk_id', 'fn'], right_on = ['klikk_name', 'klikk_id', 'fn'])


In [107]:
raw_sets[ (raw_sets.klikk_id == 0) &
        (raw_sets.klikk_elem == 'jó')]

,klikk_name,klikk_id,klikk_elem,klikk_group_id,source_file,mn,fn,count


In [108]:
raw_sets_aggs = raw_sets[['klikk_name', 'klikk_id', 'fn', 'mn']].groupby(by=['klikk_name', 'klikk_id', 'fn']).count().reset_index()


In [159]:
# Minden klikk, minden csoportjában pontosan egyszer fordul elő
klikk_leiro_fn = raw_sets_aggs[raw_sets_aggs.mn == 1]
klikk_leiro_fn

,klikk_name,klikk_id,fn,mn


In [160]:
# Most visszacsatoljuk, hogy tudjuk, hogy melyik főnév melyik csoportban van benne

klikk_leirok = raw_sets.merge(klikk_leiro_fn[['klikk_name', 'klikk_id', 'fn']], how = 'inner', 
               left_on = ['klikk_name', 'klikk_id', 'fn'], right_on = ['klikk_name', 'klikk_id', 'fn'])

klikk_leirok.to_excel('../results/klikk_leiro.xlsx', index=False)
